# Generation of files for media preparation using biomek

This notebook generates several files used in an automated media optimization project. Given a file with stock concentrations and a file with target concentrations (recommended for example by ART), this notebooks generates transfer volumes needed to achieve those target concentrations. Transfer volumes are provided as a table for all components and destination wells, but also as files required for a Biomek run. The notebook also generates a file with instructions on how to prepare a plate with stock solutions.

## Inputs and outputs

### Required files to run this notebook:
   - `standard_recipe_concentrations.csv` - this file will not change over the course of a particular project
   
   - `stock_concentrations.csv` - this file will not change over the course of a particular project
   
   - `../data/DBTLX/target_concentrations.csv` - this is an output from an ART run and it will change at every DBTL cycle
   
   - `24-well_stock_plate_high.csv`, `24-well_stock_plate_low.csv`, `24-well_stock_plate_fresh.csv` - instructions on how to prepare the source plates
     

### Files generated by running this notebook:
   
   - `dest_volumes.csv` - volumes of all components in each of the destination wells
   
   - `media_descriptions.csv` - complete list of media designs in terms of concentrations for each each well
   
   
   
The files that need to be uploaded to the Biomek, following this particular order:
   
   - `P200_water.csv`
   
   - `P20_water.csv`
         
   - `P20_kan.csv` 
   
   - `P200_components.csv` 
   
   - `P20_components.csv` 
  
   - `P20_culture.csv` 
   
   
Note that all these different files are needed as biomek needs a separate one for different types of source/destination plate and tips. Also, it cannot follow the order of operations within a file so we have to enforce an order using different files (e.g. first a file with kan transfers, then a file with components transfers).
    
The files are stored in the user defined directory. 

## Setup

Importing needed libraries:

In [462]:
import os
import sys
sys.path.append('../media_compiler')

import pandas as pd
import numpy as np

from core import find_volumes, find_volumes_bulk


## User parameters

In [463]:
CYCLE = '5.1'

user_params = {
    'stock_conc_file': '../flaviolin data/stock_concentrations.csv',
    'standard_media_file': '../flaviolin data/standard_recipe_concentrations.csv',
    'stock_plate_high_file': '../flaviolin data/24-well_stock_plate_high_extended.csv',
    'stock_plate_low_file': '../flaviolin data/24-well_stock_plate_low_extended.csv',
    'stock_plate_fresh_file': '../flaviolin data/24-well_stock_plate_fresh_extended.csv',
    'target_conc_file': f'../flaviolin data/DBTL{CYCLE}/target_concentrations.csv',
    'output_path': f'../flaviolin data/DBTL{CYCLE}',  # Path for output files
    'well_volume': 1500,            # Total volume of the media content in the destination well
    'tips': ['f20', 'f200'],        # Choose available tips from f20, s20, f50, s50, f200, s200
    'min_transfer_volume': 5.,      # Minimal transfer volume of the liquid handler
    'culture_factor': 100           # Dilution factor for culture, e.g. 100x, 1000x

}

Setup tips

In [464]:
df_tips = pd.DataFrame([['f20', 36],
                        ['s20', 72],
                        ['f50', 45],
                        ['s50', 81],
                        ['f200', 190],
                        ['s200', 171]],
                       columns=['Tips', 'Max Volume'],
                       ).set_index('Tips')


In [465]:
max_tip_volume = max([df_tips.loc[tip]['Max Volume'] for tip in user_params['tips']])


In [466]:
df_tips

,Max Volume
Tips,
f20,36
s20,72
f50,45
s50,81
f200,190
s200,171


In [467]:
max_tip_volume

190

Load the standard media recipe

In [468]:
df_stand = pd.read_csv(user_params['standard_media_file']).set_index("Component")

In [469]:
# df_stand

Load the stock concentrations and stock plate layouts:

In [470]:
df_stock = pd.read_csv(user_params['stock_conc_file'])
df_stock = df_stock.set_index("Component")
df_stock_plate_high = pd.read_csv(user_params['stock_plate_high_file'])
df_stock_plate_low = pd.read_csv(user_params['stock_plate_low_file'])
df_stock_plate_fresh = pd.read_csv(user_params['stock_plate_fresh_file'])


In [471]:
df_stock_plate_fresh

,Well,Component,Concentration
0,A1,Culture,NaN
1,B1,FeSO4[mM],0.3
2,C1,FeSO4[mM],6.0


In [472]:
df_stock

,Low Concentration,High Concentration,Dilution Factor
Component,,,
MOPS[mM],1700.0000,1700.000,1.000000
Tricine[mM],400.0000,400.000,1.000000
H3BO3[mM],0.1200,2.400,20.000000
Glucose[mM],3000.0000,3000.000,1.000000
K2SO4[mM],3.0000,43.500,14.500000
K2HPO4[mM],79.2000,396.000,5.000000
FeSO4[mM],0.3000,6.000,20.000000
NH4Cl[mM],1920.0000,1900.000,0.989583
MgCl2[mM],7.8000,15.600,2.000000


Reformat values from string to float in the `df_stock` dataframe:

In [473]:
# df_stock.loc['Kan'] = [300., 300., 1.]
# df_stock["High Concentration"] = df_stock["High Concentration"].astype(float)
# df_stock["Low Concentration"] = df_stock["Low Concentration"].astype(float)

In [474]:
df_stock

,Low Concentration,High Concentration,Dilution Factor
Component,,,
MOPS[mM],1700.0000,1700.000,1.000000
Tricine[mM],400.0000,400.000,1.000000
H3BO3[mM],0.1200,2.400,20.000000
Glucose[mM],3000.0000,3000.000,1.000000
K2SO4[mM],3.0000,43.500,14.500000
K2HPO4[mM],79.2000,396.000,5.000000
FeSO4[mM],0.3000,6.000,20.000000
NH4Cl[mM],1920.0000,1900.000,0.989583
MgCl2[mM],7.8000,15.600,2.000000


## Read target concentrations

Read ART suggested target concentrations. Note that those are only for components which are being explored (not those that are kept constant). 

In [475]:
target_conc_file = user_params['target_conc_file']
df_target_conc = pd.read_csv(target_conc_file, index_col=0)
df_target_conc.head()

,H3BO3[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],OD340_pred,OD340_std,Label
Well,,,,,,,,,,,,,,,
A1,0.076872,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.283741,0.085632,explore
B1,0.076872,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.283741,0.085632,explore
C1,0.076872,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.283741,0.085632,explore
D1,0.074569,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.283942,0.085551,explore
E1,0.074569,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.283942,0.085551,explore


### Add fixed components and antibiotic concentrations from the standard recipe to the target concentrations dataframe

Find fixed components as those from the standard recipe that are not listed in target concentration file, which is an output from ART

In [476]:
comp_var = df_target_conc.iloc[:, :-3].columns
comp_fixed = list(df_stand.drop(comp_var).index)
print('Fixed components: ')
for comp in comp_fixed:
    df_target_conc[comp] = df_stand.at[comp, 'Concentration']
    print(f'{comp}')

Fixed components: 
MOPS[mM]
Tricine[mM]
Glucose[mM]
Kan[g/l]


Make sure the order of columns is the same as in the stock dataframe:

In [477]:
columns = df_stock.index
df_target_conc = df_target_conc[columns]

In [478]:
df_target_conc.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05
B1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05
C1,40.0,4.0,0.076872,20.0,0.673352,4.035057,0.044932,37.450241,1.085799,589.411309,0.000352,0.002470,0.005744,0.006587,0.000057,0.05
D1,40.0,4.0,0.074569,20.0,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.05
E1,40.0,4.0,0.074569,20.0,0.710400,3.865275,0.043698,37.793069,1.002822,589.640781,0.000391,0.003163,0.005799,0.012223,0.000270,0.05


Save a file with a complete list of components concentrations, describing the media (this file will lated be used to create experiment description file for EDD import).

In [479]:
final_conc_file = f"{user_params['output_path']}/media_descriptions.csv"
df_target_conc.drop(columns='Kan[g/l]').to_csv(final_conc_file)
df_target_conc.to_csv(final_conc_file)

## Calculate all transfer volumes

Also, create a dataframe with levels of stock concentrations needed to achieve those volumes, which will indicate from which source well the transfer should be made.

In [480]:
df_volumes, df_conc_level = find_volumes_bulk(
    df_stock=df_stock, 
    df_target_conc=df_target_conc,
    well_volume=user_params['well_volume'],
    min_tip_volume=user_params['min_transfer_volume'],
    culture_ratio=user_params['culture_factor'],
    verbose=1
)

Iteration 0:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Iteration 1:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Iteration 2:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Iteration 3:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Iteration 4:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Iteration 5:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Iteration 6:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Failed High + Low min vol
Failed Low
Iteration 7:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Failed High + Low min vol
Failed Low
Iteration 8:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Failed High + Low min vol
Failed Low
Iteration 9:
Compoments small: Index(['ZnSO4[mM]'], dtype='object', name='Component')
Iteration 10:
Compoments smal

In [481]:
df_volumes.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Water
Well,,,,,,,,,,,,,,,,,
A1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373
B1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373
C1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373
D1,35.294118,15.0,46.605519,10.0,24.496566,14.641194,10.924418,29.836634,96.425177,589.640781,32.550243,26.355131,144.986387,38.196197,134.951664,5.0,230.095972
E1,35.294118,15.0,46.605519,10.0,24.496566,14.641194,10.924418,29.836634,96.425177,589.640781,32.550243,26.355131,144.986387,38.196197,134.951664,5.0,230.095972


This is ok, as we don't have CaCl2 in the standard design.

In [482]:
df_conc_level.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high
B1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high
C1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high
D1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high
E1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high


Add volumes for culture

In [483]:
df_volumes['Culture'] = user_params['well_volume'] / user_params['culture_factor']
df_volumes.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Water,Culture
Well,,,,,,,,,,,,,,,,,,
A1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373,15.0
B1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373,15.0
C1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373,15.0
D1,35.294118,15.0,46.605519,10.0,24.496566,14.641194,10.924418,29.836634,96.425177,589.640781,32.550243,26.355131,144.986387,38.196197,134.951664,5.0,230.095972,15.0
E1,35.294118,15.0,46.605519,10.0,24.496566,14.641194,10.924418,29.836634,96.425177,589.640781,32.550243,26.355131,144.986387,38.196197,134.951664,5.0,230.095972,15.0


Check if sum of all volumes is equal to total well volume

In [484]:
EPS = 0.000001
assert (np.sum(df_volumes.values, axis=1) - user_params['well_volume'] <= EPS).all(), 'Sum of all volumes is not equal to total well volume!' 

### Save volumes in the destination wells to a file

In [485]:
volumes_file = f"{user_params['output_path']}/dest_volumes.csv"
df_volumes.to_csv(volumes_file)

## Add minimal volumes needed in the source plates

Define the volumes for the 24-well source plate:

In [486]:
well_volume = 9000  # including dead volume
dead_volume = 100


In [487]:
tot_vol_water = np.sum(df_volumes['Water'].values)
print(f'Total volume of water needed: {tot_vol_water:.0f} uL + dead volume')

Total volume of water needed: 20784 uL + dead volume


In [488]:
df_volumes.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Water,Culture
Well,,,,,,,,,,,,,,,,,,
A1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373,15.0
B1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373,15.0
C1,35.294118,15.0,48.045071,10.0,23.219043,15.284305,11.232995,29.565980,104.403738,589.411309,29.366916,20.583921,143.612341,20.585298,28.418591,5.0,355.976373,15.0
D1,35.294118,15.0,46.605519,10.0,24.496566,14.641194,10.924418,29.836634,96.425177,589.640781,32.550243,26.355131,144.986387,38.196197,134.951664,5.0,230.095972,15.0
E1,35.294118,15.0,46.605519,10.0,24.496566,14.641194,10.924418,29.836634,96.425177,589.640781,32.550243,26.355131,144.986387,38.196197,134.951664,5.0,230.095972,15.0


In [489]:
df_conc_level.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high
B1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high
C1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high
D1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high
E1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,low,high


In [490]:
df_stock_plate_high

,Well,Component,Concentration
0,A1,MOPS[mM],1700.000
1,B1,Tricine[mM],400.000
2,C1,H3BO3[mM],2.400
3,D1,Glucose[mM],3000.000
4,A2,K2SO4[mM],43.500
5,B2,K2HPO4[mM],396.000
6,C2,NH4Cl[mM],1900.000
7,D2,MgCl2[mM],15.600
8,A3,NaCl[mM],1500.000
9,B3,(NH4)6Mo7O24[mM],0.018


### High level

In [491]:
df_stock_plate_high['Volume [uL]'] = None

for i in range(len(df_stock_plate_high)):
    comp = df_stock_plate_high.iloc[i]['Component']
    stock_level = 'high'
    if comp in df_conc_level.columns:
        tot_vol_comp = np.sum(
            df_volumes[df_conc_level[comp]==stock_level][comp].values
        )
        df_stock_plate_high.loc[i, 'Volume [uL]'] = np.round(tot_vol_comp)
        print(df_stock_plate_high)
    
df_stock_plate_high


   Well         Component  Concentration Volume [uL]
0    A1          MOPS[mM]       1700.000      1694.0
1    B1       Tricine[mM]        400.000        None
2    C1         H3BO3[mM]          2.400        None
3    D1       Glucose[mM]       3000.000        None
4    A2         K2SO4[mM]         43.500        None
5    B2        K2HPO4[mM]        396.000        None
6    C2         NH4Cl[mM]       1900.000        None
7    D2         MgCl2[mM]         15.600        None
8    A3          NaCl[mM]       1500.000        None
9    B3  (NH4)6Mo7O24[mM]          0.018        None
10   C3         CoCl2[mM]          0.180        None
11   D3         CuSO4[mM]          0.060        None
12   A4         MnSO4[mM]          0.480        None
13   B4         ZnSO4[mM]          0.300        None
14   C4          Kan[g/l]         15.000        None
   Well         Component  Concentration Volume [uL]
0    A1          MOPS[mM]       1700.000      1694.0
1    B1       Tricine[mM]        400.000      

,Well,Component,Concentration,Volume [uL]
0,A1,MOPS[mM],1700.000,1694.0
1,B1,Tricine[mM],400.000,720.0
2,C1,H3BO3[mM],2.400,1051.0
3,D1,Glucose[mM],3000.000,480.0
4,A2,K2SO4[mM],43.500,1066.0
5,B2,K2HPO4[mM],396.000,743.0
6,C2,NH4Cl[mM],1900.000,774.0
7,D2,MgCl2[mM],15.600,3710.0
8,A3,NaCl[mM],1500.000,22406.0
9,B3,(NH4)6Mo7O24[mM],0.018,1013.0


Are the total volumes smaller than well volume of the plate? If not, assign additional wells.

In [492]:
df_new_wells = pd.DataFrame(columns=df_stock_plate_high.columns)
ind_drop = []
num_all_wells = len(df_stock_plate_high)
print(df_stock_plate_high)

for i in range(len(df_stock_plate_high)-2):
    tot_volume = df_stock_plate_high.iloc[i]['Volume [uL]']
    if tot_volume + dead_volume > well_volume:
        ind_drop = ind_drop + [i]
        comp = df_stock_plate_high.iloc[i]['Component']
        conc_level = 'high'
        num_wells_needed = int(np.ceil(tot_volume / well_volume))
        print(f'We need {num_wells_needed} well(s) for {comp} with {conc_level} concentration')
        # Include additional wells needed
        indices = [i]
        indices.extend(range(num_all_wells, num_all_wells+num_wells_needed-1))
        num_all_wells += num_wells_needed
        volumes = []
        volume_left = tot_volume + num_wells_needed*dead_volume
        while volume_left > dead_volume:
            volumes = volumes + [min(volume_left, well_volume)] 
            volume_left = volume_left - volumes[-1]
            
        d = {
             'Component' : pd.Series(comp, index=indices),
             'Concentration': pd.Series(df_stock_plate_high.iloc[i]['Concentration'], index =indices),
             'Volume [uL]': pd.Series(volumes, index=indices)}
        
        df = pd.DataFrame(d)
        print(df)
        df_new_wells = pd.concat([df_new_wells,df], ignore_index=False)        
        print(df_new_wells)
    else:
        df_stock_plate_high.at[i, 'Volume [uL]'] += dead_volume
        print('.', end='')
print('Finished.')

df_stock_plate_high.drop(ind_drop, inplace=True)
df_stock_plate_high = pd.concat([df_stock_plate_high,df_new_wells], ignore_index=False)
df_stock_plate_high = df_stock_plate_high.dropna(subset=['Volume [uL]'])
df_stock_plate_high = df_stock_plate_high.sort_index().reset_index(drop=True) 


   Well         Component  Concentration Volume [uL]
0    A1          MOPS[mM]       1700.000      1694.0
1    B1       Tricine[mM]        400.000       720.0
2    C1         H3BO3[mM]          2.400      1051.0
3    D1       Glucose[mM]       3000.000       480.0
4    A2         K2SO4[mM]         43.500      1066.0
5    B2        K2HPO4[mM]        396.000       743.0
6    C2         NH4Cl[mM]       1900.000       774.0
7    D2         MgCl2[mM]         15.600      3710.0
8    A3          NaCl[mM]       1500.000     22406.0
9    B3  (NH4)6Mo7O24[mM]          0.018      1013.0
10   C3         CoCl2[mM]          0.180       914.0
11   D3         CuSO4[mM]          0.060      3598.0
12   A4         MnSO4[mM]          0.480       729.0
13   B4         ZnSO4[mM]          0.300        19.0
14   C4          Kan[g/l]         15.000       240.0
........We need 3 well(s) for NaCl[mM] with high concentration
   Component  Concentration  Volume [uL]
8   NaCl[mM]         1500.0       9000.0
15  NaC

In [493]:
ind_drop

[8]

In [494]:
df_stock_plate_high


,Well,Component,Concentration,Volume [uL]
0,A1,MOPS[mM],1700.000,1794.0
1,B1,Tricine[mM],400.000,820.0
2,C1,H3BO3[mM],2.400,1151.0
3,D1,Glucose[mM],3000.000,580.0
4,A2,K2SO4[mM],43.500,1166.0
5,B2,K2HPO4[mM],396.000,843.0
6,C2,NH4Cl[mM],1900.000,874.0
7,D2,MgCl2[mM],15.600,3810.0
8,NaN,NaCl[mM],1500.000,9000.0
9,B3,(NH4)6Mo7O24[mM],0.018,1113.0


Assign well names:

In [495]:
well_rows = 'ABCD'
well_columns = '123456'

In [496]:
num_source_wells = len(df_stock_plate_high) 
    
well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate_high['Well'] = well_names
df_stock_plate_high = df_stock_plate_high.set_index("Well")
df_stock_plate_high

,Component,Concentration,Volume [uL]
Well,,,
A1,MOPS[mM],1700.000,1794.0
B1,Tricine[mM],400.000,820.0
C1,H3BO3[mM],2.400,1151.0
D1,Glucose[mM],3000.000,580.0
A2,K2SO4[mM],43.500,1166.0
B2,K2HPO4[mM],396.000,843.0
C2,NH4Cl[mM],1900.000,874.0
D2,MgCl2[mM],15.600,3810.0
A3,NaCl[mM],1500.000,9000.0


### Low level

In [497]:
df_stock_plate_low['Volume [uL]'] = None

for i in range(len(df_stock_plate_low)):
    comp = df_stock_plate_low.iloc[i]['Component']
    stock_level = 'low'
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate_low.iloc[i, df_stock_plate_low.columns.get_loc('Volume [uL]')] = np.round(tot_vol_comp)

df_stock_plate_low

,Well,Component,Concentration,Volume [uL]
0,A1,H3BO3[mM],0.1200,160.0
1,B1,K2SO4[mM],3.0000,0.0
2,C1,K2HPO4[mM],79.2000,75.0
3,D1,MgCl2[mM],7.8000,0.0
4,A2,(NH4)6Mo7O24[mM],0.0009,152.0
5,B2,CoCl2[mM],0.0090,140.0
6,C2,CuSO4[mM],0.0030,142.0
7,D2,MnSO4[mM],0.0240,848.0
8,A3,ZnSO4[mM],0.0030,9045.0


In [498]:
df_new_wells = pd.DataFrame(columns=df_stock_plate_low.columns)
ind_drop = []
num_all_wells = len(df_stock_plate_high)
for i in range(len(df_stock_plate_low)):
    tot_volume = df_stock_plate_low.iloc[i]['Volume [uL]']
    if tot_volume + dead_volume > well_volume:
        ind_drop = ind_drop + [i]
        comp = df_stock_plate_low.iloc[i]['Component']
        conc_level = 'low'
        num_wells_needed = int(np.ceil(tot_volume / well_volume))
        print(f'We need {num_wells_needed} well(s) for {comp} with {conc_level} concentration')
        # Include additional wells needed
        indices = [i]
        indices.extend(range(num_all_wells, num_all_wells+num_wells_needed-1))
        num_all_wells += num_wells_needed
        volumes = []
        volume_left = tot_volume + num_wells_needed*dead_volume
        while volume_left > dead_volume:
            volumes = volumes + [min(volume_left, well_volume)] 
            volume_left = volume_left - volumes[-1]
            
        d = {'Component' : pd.Series(comp, index=indices),
             'Concentration': pd.Series(df_stock_plate_low.iloc[0]['Concentration'], index =indices),
             'Volume [uL]': pd.Series(volumes, index=indices)}

        df = pd.DataFrame(d)
        df_new_wells = pd.concat([df_new_wells,df], ignore_index=False)        

    else:
        df_stock_plate_low.at[i,'Volume [uL]'] += dead_volume
        print('.', end='')
print('Finished.')

df_stock_plate_low.drop(ind_drop, inplace=True)
df_stock_plate_low = pd.concat([df_stock_plate_low,df_new_wells], ignore_index=False)
df_stock_plate_low = df_stock_plate_low.sort_index().reset_index(drop=True) 


........We need 2 well(s) for ZnSO4[mM] with low concentration
Finished.


Assign well names:

In [499]:
df_stock_plate_low = df_stock_plate_low.set_index("Well")
df_stock_plate_low

,Component,Concentration,Volume [uL]
Well,,,
A1,H3BO3[mM],0.1200,260.0
B1,K2SO4[mM],3.0000,100.0
C1,K2HPO4[mM],79.2000,175.0
D1,MgCl2[mM],7.8000,100.0
A2,(NH4)6Mo7O24[mM],0.0009,252.0
B2,CoCl2[mM],0.0090,240.0
C2,CuSO4[mM],0.0030,242.0
D2,MnSO4[mM],0.0240,948.0
NaN,ZnSO4[mM],0.1200,9000.0


Assign well names

In [500]:
well_rows = 'ABCD'
well_columns = '123456'

In [501]:
num_source_wells = len(df_stock_plate_low) 
    
well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate_low['Well'] = well_names
df_stock_plate_low = df_stock_plate_low.set_index("Well")
df_stock_plate_low

,Component,Concentration,Volume [uL]
Well,,,
A1,H3BO3[mM],0.1200,260.0
B1,K2SO4[mM],3.0000,100.0
C1,K2HPO4[mM],79.2000,175.0
D1,MgCl2[mM],7.8000,100.0
A2,(NH4)6Mo7O24[mM],0.0009,252.0
B2,CoCl2[mM],0.0090,240.0
C2,CuSO4[mM],0.0030,242.0
D2,MnSO4[mM],0.0240,948.0
A3,ZnSO4[mM],0.1200,9000.0


### Fresh components

In [502]:
df_stock_plate_fresh['Volume [uL]'] = None

comp = 'FeSO4[mM]'
for i, stock_level in enumerate(list(['low', 'high'])):
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate_fresh.loc[i+1, 'Volume [uL]'] = np.round(tot_vol_comp)

# Culture
tot_vol_culture = np.sum(df_volumes['Culture'].values) 
df_stock_plate_fresh.loc[0, 'Volume [uL]'] = np.round(tot_vol_culture)

df_stock_plate_fresh = df_stock_plate_fresh.set_index("Well")
df_stock_plate_fresh


,Component,Concentration,Volume [uL]
Well,,,
A1,Culture,NaN,720.0
B1,FeSO4[mM],0.3,160.0
C1,FeSO4[mM],6.0,616.0


Save source plate instructions:

In [503]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_high.csv"
df_stock_plate_high.to_csv(stock_plate_file)

In [504]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_low.csv"
df_stock_plate_low.to_csv(stock_plate_file)

In [505]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_fresh.csv"
df_stock_plate_fresh.to_csv(stock_plate_file)

## Define transfers for biomek

In [506]:
column_names = [
    "Source Position",
    "Source Well",
    "Destination Position",
    "Destination Well",
    "Transfer Volume [uL]"
]

### Create water transfers

In [507]:
P20_water = pd.DataFrame(columns=column_names)
P200_water = P20_water.copy()

i = 0  # counter for index

comp = 'Water'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_water.at[i, "Destination Well"] = dest_well
        P20_water.at[i, "Transfer Volume [uL]"] = vol
        i += 1

    # P200 transfer
    else:
        # Divide the transfer in parts until whole volume is transfered
        # How many transfers
        num_transfers = int(np.ceil(vol / max_tip_volume))
        transf_vol = vol / num_transfers
        for trans_num in range(num_transfers):
            P200_water.at[i, "Destination Well"] = dest_well
            P200_water.at[i, "Transfer Volume [uL]"] = transf_vol
            i += 1

P20_water["Source Position"] = "P3" 
P20_water["Source Well"] = "A1"  # reservoir plate
P20_water["Destination Position"] = "P2" 
P200_water["Source Position"] = "P3" 
P200_water["Source Well"] = "A1"
P200_water["Destination Position"] = "P2" 

In [508]:
P200_water

,Source Position,Source Well,Destination Position,Destination Well,Transfer Volume [uL]
0,P3,A1,P2,A1,177.988187
1,P3,A1,P2,A1,177.988187
2,P3,A1,P2,B1,177.988187
3,P3,A1,P2,B1,177.988187
4,P3,A1,P2,C1,177.988187
...,...,...,...,...,...
127,P3,A1,P2,F8,188.242341
128,P3,A1,P2,F8,188.242341
129,P3,A1,P2,F8,188.242341
130,P3,A1,P2,F8,188.242341


### Create kan transfers

In [509]:
P20_kan = pd.DataFrame(columns=column_names)
P200_kan = P20_kan.copy()

i = 0  # counter for index

comp = 'Kan[g/l]'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_kan.at[i, "Destination Well"] = dest_well
        P20_kan.at[i, "Transfer Volume [uL]"] = vol
        i += 1

    # P200 transfer
    else:
        # Divide the transfer in parts until whole volume is transfered
        # How many transfers
        num_transfers = int(np.ceil(vol / max_tip_volume))
        if num_transfers > 1:
            print(num_transfers)
        transf_vol = vol / num_transfers
        for trans_num in range(num_transfers):
            P200_kan.at[i, "Destination Well"] = dest_well
            P200_kan.at[i, "Transfer Volume [uL]"] = transf_vol
            i += 1

P20_kan["Source Position"] = "P1" 
P20_kan["Source Well"] = df_stock_plate_high[
    df_stock_plate_high["Component"]==comp
].index[0]  
P20_kan["Destination Position"] = "P2"
P200_kan["Source Position"] = "P1" 
P200_kan["Source Well"] = df_stock_plate_high[
    df_stock_plate_high["Component"]==comp
].index[0]  
P200_kan["Destination Position"] = "P2"


### Create culture transfers

Note that we assume here the volumes for culture are small enough for p20 pipette, if that is not the case you need to adjust.

In [510]:
P20_culture = pd.DataFrame(columns=column_names)

comp = 'Culture'
vol = df_volumes[comp][0]
    
for i, dest_well in enumerate(df_volumes.index):
    
    P20_culture.at[i, "Destination Well"] = dest_well
    P20_culture.at[i, "Transfer Volume [uL]"] = vol

# DBTL 1
# P20_culture["Source Position"] = "P4" 
# P20_culture["Source Well"] = df_stock_plate_low[
#     df_stock_plate_low["Component"]==comp
# ].index[0]

P20_culture["Source Position"] = "P5" 
P20_culture["Source Well"] = df_stock_plate_fresh[
    df_stock_plate_fresh["Component"]==comp
].index[0]
P20_culture["Destination Position"] = "P2" 


### Create component transfers

Create a column to track the current volume in the wells of the source plate:

In [511]:
df_stock_plate_high['Current Volume'] = well_volume
df_stock_plate_low['Current Volume'] = well_volume
df_stock_plate_fresh['Current Volume'] = well_volume

In [512]:
P20_components = pd.DataFrame(columns=column_names) 
P200_components = P20_components.copy()

In [513]:
df_volumes.columns

Index(['MOPS[mM]', 'Tricine[mM]', 'H3BO3[mM]', 'Glucose[mM]', 'K2SO4[mM]',
       'K2HPO4[mM]', 'FeSO4[mM]', 'NH4Cl[mM]', 'MgCl2[mM]', 'NaCl[mM]',
       '(NH4)6Mo7O24[mM]', 'CoCl2[mM]', 'CuSO4[mM]', 'MnSO4[mM]', 'ZnSO4[mM]',
       'Kan[g/l]', 'Water', 'Culture'],
      dtype='object')

In [514]:
df_stock_plate_high


,Component,Concentration,Volume [uL],Current Volume
Well,,,,
A1,MOPS[mM],1700.000,1794.0,9000
B1,Tricine[mM],400.000,820.0,9000
C1,H3BO3[mM],2.400,1151.0,9000
D1,Glucose[mM],3000.000,580.0,9000
A2,K2SO4[mM],43.500,1166.0,9000
B2,K2HPO4[mM],396.000,843.0,9000
C2,NH4Cl[mM],1900.000,874.0,9000
D2,MgCl2[mM],15.600,3810.0,9000
A3,NaCl[mM],1500.000,9000.0,9000


In [515]:
df_stock_plate_low


,Component,Concentration,Volume [uL],Current Volume
Well,,,,
A1,H3BO3[mM],0.1200,260.0,9000
B1,K2SO4[mM],3.0000,100.0,9000
C1,K2HPO4[mM],79.2000,175.0,9000
D1,MgCl2[mM],7.8000,100.0,9000
A2,(NH4)6Mo7O24[mM],0.0009,252.0,9000
B2,CoCl2[mM],0.0090,240.0,9000
C2,CuSO4[mM],0.0030,242.0,9000
D2,MnSO4[mM],0.0240,948.0,9000
A3,ZnSO4[mM],0.1200,9000.0,9000


In [516]:
i = 0  # counter for index

components = list(df_volumes.columns.drop(['Kan[g/l]', 'Water', 'Culture']))

for comp in components:
    
    for dest_well in df_volumes.index:
        vol = df_volumes.at[dest_well, comp]
        conc_level = df_conc_level.at[dest_well, comp]
        
        # Find which plate, well we need to use as source
        if comp == 'FeSO4[mM]':
            plate_position = "P5"
            df_stock_plate = df_stock_plate_fresh
            source_well = "B1" if conc_level == 'low' else "C1"
        else:
            if conc_level == 'high':
                plate_position = "P1" 
                df_stock_plate = df_stock_plate_high
            else:
                plate_position = "P4"
                df_stock_plate = df_stock_plate_low

            # Find the source well in the stock plate with enough volume 
            # Find all wells with this component
            source_wells = df_stock_plate[
                (df_stock_plate["Component"]==comp)
            ].index

            # Find indices where these wells have volume larger than transfer volume
            ind_vol = np.where(df_stock_plate.loc[source_wells, 'Current Volume']  > vol + dead_volume)[0]
            # Assign as the source well the first one with enough volume
            print(comp)
            print('----')
            print(df_stock_plate.loc[source_wells, :])
            print('----')
            print('source wells', source_wells)
            print('----')
            print('vol + dead vol', vol + dead_volume)
            print('----')
            print('ind_vol, np.where(df_stock_plate.loc[source_wells, Current Volume]  > vol + dead_volume)[0]', ind_vol)
            source_well = source_wells[ind_vol[0]]    
        
        # P20 transfer
        if vol < 30:
            P20_components.at[i, "Source Well"] = source_well
            P20_components.at[i, "Destination Well"] = dest_well
            P20_components.at[i, "Transfer Volume [uL]"] = vol
            P20_components.at[i, "Source Position"] = plate_position
            i += 1

        # P200 transfer
        else:
            # Divide the transfer in parts until whole volume is transfered
            # How many transfers
            num_transfers = int(np.ceil(vol / max_tip_volume))
            transf_vol = vol / num_transfers
            for trans_num in range(num_transfers):
                P200_components.at[i, "Source Well"] = source_well
                P200_components.at[i, "Destination Well"] = dest_well
                P200_components.at[i, "Transfer Volume [uL]"] = transf_vol
                P200_components.at[i, "Source Position"] = plate_position
                i += 1
                    
        # Update the current volume in the source well
        df_stock_plate.loc[source_well, 'Current Volume'] -= vol

P20_components["Destination Position"] = "P2" 
P200_components["Destination Position"] = "P2" 

MOPS[mM]
----
     Component  Concentration Volume [uL]  Current Volume
Well                                                     
A1    MOPS[mM]         1700.0      1794.0            9000
----
source wells Index(['A1'], dtype='object', name='Well')
----
vol + dead vol 135.29411764705867
----
ind_vol, np.where(df_stock_plate.loc[source_wells, Current Volume]  > vol + dead_volume)[0] [0]
MOPS[mM]
----
     Component  Concentration Volume [uL]  Current Volume
Well                                                     
A1    MOPS[mM]         1700.0      1794.0     8964.705882
----
source wells Index(['A1'], dtype='object', name='Well')
----
vol + dead vol 135.29411764705867
----
ind_vol, np.where(df_stock_plate.loc[source_wells, Current Volume]  > vol + dead_volume)[0] [0]
MOPS[mM]
----
     Component  Concentration Volume [uL]  Current Volume
Well                                                     
A1    MOPS[mM]         1700.0      1794.0     8929.411765
----
source wells Index(['A1'], dt

In [517]:
P20_components.head(5)

,Source Position,Source Well,Destination Position,Destination Well,Transfer Volume [uL]
48,P1,B1,P2,A1,15.0
49,P1,B1,P2,B1,15.0
50,P1,B1,P2,C1,15.0
51,P1,B1,P2,D1,15.0
52,P1,B1,P2,E1,15.0


## Calculate number of transfers and tips needed

In [518]:
num_transf_water_p200 = len(P200_water)
num_transf_water_p20 = len(P20_water)
num_transf_comp_p20 = len(P20_components)
num_transf_comp_p200 = len(P200_components)
num_transf_culture = len(P20_culture)
num_transf_kan_p20 = len(P20_kan)
num_transf_kan_p200 = len(P200_kan)
num_mixing = 1  # mixing the culture well 
num_total_transf = num_transf_water_p200 + num_transf_water_p20 + num_transf_comp_p20 + num_transf_comp_p200+num_transf_kan_p20+num_transf_kan_p200+num_transf_culture+num_mixing
print(f'Number of transfers:')
print(f'\t Water (p200): {num_transf_water_p200}')
print(f'\t Water (p20): {num_transf_water_p20}')
print(f'\t Kan (p20): {num_transf_kan_p20}')
print(f'\t Kan (p200): {num_transf_kan_p200}')
print(f'\t Components (p200): {num_transf_comp_p200}')
print(f'\t Components (p20): {num_transf_comp_p20}')
print(f'\t Culture (p20): {num_transf_culture}')
print(f'\t Mixing (p20): {num_mixing}')
print(f'Total number of transfers: {num_total_transf}')

Number of transfers:
	 Water (p200): 132
	 Water (p20): 0
	 Kan (p20): 48
	 Kan (p200): 0
	 Components (p200): 429
	 Components (p20): 423
	 Culture (p20): 48
	 Mixing (p20): 1
Total number of transfers: 1081


Calculate number of tip boxes needed:

In [519]:
print(f'This protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200 
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_comp_p20 + num_transf_kan_p20 + num_mixing + num_transf_culture
full_box, rest = divmod(num_tips_200, 96)
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')


This protocol requires:
	5 box(es) of p200 tips
		4 box(es) + 52 tips
	6 box(es) of p20 tips
		5 box(es) + 40 tips


Introduce a pause in the biomek method after p200 components transfers.

In [520]:
print(f'The first part of the protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200 
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_kan_p20 

full_box, rest = divmod(num_tips_200, 96)
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')

The first part of the protocol requires:
	5 box(es) of p200 tips
		4 box(es) + 52 tips
	1 box(es) of p20 tips
		0 box(es) + 48 tips


For the next step we need:

In [521]:
print(f'The second part of the protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_20 = num_transf_comp_p20  + num_mixing + num_transf_culture
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')

The second part of the protocol requires:
	5 box(es) of p20 tips
		4 box(es) + 88 tips


### Save biomek files

In [522]:
biomek_files_dir = f"{user_params['output_path']}/biomek_files/"
os.makedirs(biomek_files_dir, exist_ok=True)

P200_water_file = f"{biomek_files_dir}/P200_water.csv"
P20_water_file = f"{biomek_files_dir}/P20_water.csv"
P20_kan_file = f"{biomek_files_dir}/P20_kan.csv"
P200_components_file = f"{biomek_files_dir}/P200_components.csv"
P20_components_file = f"{biomek_files_dir}/P20_components.csv"
P20_culture_file = f"{biomek_files_dir}/P20_culture.csv"

P200_water.to_csv(P200_water_file, index=False)
P20_water.to_csv(P20_water_file, index=False)
P20_kan.to_csv(P20_kan_file, index=False)
P200_components.to_csv(P200_components_file, index=False)
P20_components.to_csv(P20_components_file, index=False)
P20_culture.to_csv(P20_culture_file, index=False)

In [523]:
P200_water['order'] = 1
P20_water['order'] = 1
P20_kan['order'] = 2
P200_components['order'] = 3
P20_components['order'] = 4
P20_culture['order'] = 5

hamilton_transfers_tmp = pd.concat([P200_water, P20_water, P20_kan, P200_components, P20_components, P20_culture], axis=0).reset_index()
hamilton_transfers_tmp.head()

,index,Source Position,Source Well,Destination Position,Destination Well,Transfer Volume [uL],order
0,0,P3,A1,P2,A1,177.988187,1
1,1,P3,A1,P2,A1,177.988187,1
2,2,P3,A1,P2,B1,177.988187,1
3,3,P3,A1,P2,B1,177.988187,1
4,4,P3,A1,P2,C1,177.988187,1


In [524]:
hamilton_transfers = pd.DataFrame()
hamilton_transfers

""


In [525]:
hamilton_transfers['order'] = hamilton_transfers_tmp['order']
hamilton_transfers['Source_Plate'] = hamilton_transfers_tmp['Source Position']
hamilton_transfers['Source_Well'] = hamilton_transfers_tmp['Source Well']
hamilton_transfers['Dest_Plate'] = hamilton_transfers_tmp['Destination Position']
hamilton_transfers['Dest_Well'] = hamilton_transfers_tmp['Destination Well']
hamilton_transfers['Transfer_Vol'] = hamilton_transfers_tmp['Transfer Volume [uL]']
hamilton_transfers.head()

,order,Source_Plate,Source_Well,Dest_Plate,Dest_Well,Transfer_Vol
0,1,P3,A1,P2,A1,177.988187
1,1,P3,A1,P2,A1,177.988187
2,1,P3,A1,P2,B1,177.988187
3,1,P3,A1,P2,B1,177.988187
4,1,P3,A1,P2,C1,177.988187


In [526]:
hamilton_files_dir = f"{user_params['output_path']}/hamilton_files/"
os.makedirs(hamilton_files_dir, exist_ok=True)

hamilton_transfers_file = f"{hamilton_files_dir}/all_transfers.xlsx"


hamilton_transfers.to_excel(hamilton_transfers_file, index=False)
